# Final Model - Random Forest

In [381]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin, OneToOneFeatureMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold, cross_val_score
from sklearn.metrics import (
    make_scorer,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score
)
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import optuna
from sklearn.utils.validation import check_is_fitted

In [382]:
url = "https://raw.githubusercontent.com/CarsonShively/Heart-Failure/refs/heads/main/data/heart_failure.csv"
df = pd.read_csv(url)
df.drop_duplicates(inplace=True)

X = df.drop('DEATH_EVENT', axis=1)
y = df['DEATH_EVENT']

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=99
)

# Feature engineering
I explored several engineered features based on clinical relevance and interaction potential. After systematic testing, only the features that consistently improved cross-validation metrics were retained.

In [383]:
class FeatureEngineer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self._transform_output = None

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        X['ef_to_creatinine'] = X['ejection_fraction'] / X['serum_creatinine']
        X['ef_drop_per_age'] = (100 - X['ejection_fraction']) / X['age']
        X['ef_per_time'] = X['ejection_fraction'] / X['time']
        X['time_x_ef'] = X['time'] * X['ejection_fraction']
        X['creatinine_x_ef'] = X['serum_creatinine'] * X['ejection_fraction']
        X['time_x_creatinine'] = X['time'] * X['serum_creatinine']

        X.replace([np.inf, -np.inf], np.nan, inplace=True)

        if self._transform_output == "pandas":
            return pd.DataFrame(X, columns=X.columns, index=X.index)
        else:
            return X

    def set_output(self, transform=None):
        self._transform_output = transform
        return self

FE_pipeline = Pipeline(steps=[
    ('FE', FeatureEngineer()),
])
FE_pipeline.set_output(transform="pandas")
_ = FE_pipeline

# Binary Features
 I evaluated multiple strategies for handling missing values, and imputing with the mode consistently yielded the best performance. After imputation, I filtered out any constant columns to prevent them from introducing noise or reducing model generalizability.

In [384]:
binary_cols = [
    'anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking'
]

class BinaryInt64Cleaner(BaseEstimator, TransformerMixin, OneToOneFeatureMixin):
    def __init__(self, binary_cols):
        self.binary_cols = binary_cols

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        for col in self.binary_cols:
            X[col] = pd.to_numeric(X[col], errors='coerce').astype('Int64')
            X[col] = X[col].where(X[col].isin([0, 1]), pd.NA)
        return X


class BinaryImputer(BaseEstimator, TransformerMixin, OneToOneFeatureMixin):
    def __init__(self, binary_cols):
        self.binary_cols = binary_cols

    def fit(self, X, y=None):
        self.modes_ = {col: X[col].mode(dropna=True)[0] for col in self.binary_cols}
        return self

    def transform(self, X):
        X = X.copy()
        for col in self.binary_cols:
            X[col] = X[col].fillna(self.modes_[col])
        return X

class ConstantBinaryDropper(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        X = X.copy()
        self.constant_features_ = [col for col in self.columns if X[col].nunique(dropna=True) == 1]

        if self.constant_features_:
            print(f"Dropping constant features: {self.constant_features_}")
        else:
            print("No constant features dropped.")

        self.feature_names_out_ = [col for col in self.columns if col not in self.constant_features_]
        return self

    def transform(self, X):
        X = X.copy()
        return X.drop(columns=self.constant_features_, errors='ignore')

    def get_feature_names_out(self, input_features=None):
        return self.feature_names_out_


binary_pipeline = Pipeline(steps=[
    ('cleaner', BinaryInt64Cleaner(binary_cols=binary_cols)),
    ('imputer', BinaryImputer(binary_cols=binary_cols)),
    ('constant_cols', ConstantBinaryDropper(columns=binary_cols))
])
binary_pipeline.set_output(transform="pandas")
_ = binary_pipeline

# Numeric features

I first checked for low-variance features to reduce noise and eliminate uninformative inputs. I then checked for highly correlated feature pairs to simplify the dataset and minimize redundancy.

In [385]:
numeric_features = [
    'age',
    'creatinine_phosphokinase',
    'ejection_fraction',
    'platelets',
    'serum_creatinine',
    'serum_sodium',
    'time',
    'ef_to_creatinine',
    'ef_drop_per_age',
    'ef_per_time',
    'time_x_ef',
    'creatinine_x_ef',
    'time_x_creatinine',
]


class CoerceToFloat(BaseEstimator, TransformerMixin, OneToOneFeatureMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        for col in self.columns:
            X[col] = X[col].astype(float)
        return X

class NumericImputer(BaseEstimator, TransformerMixin, OneToOneFeatureMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        X = X.copy()
        self.medians_ = {
            col: X[col].median(skipna=True)
            for col in self.columns
        }
        return self

    def transform(self, X):
        X = X.copy()
        for col in self.columns:
            X[col] = X[col].fillna(self.medians_[col])
        return X

class LowVarianceDropper(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.0):
        self.threshold = threshold

    def fit(self, X, y=None):
        X = X.copy()
        variances = X.var()
        self.low_variance_features_ = variances[variances <= self.threshold].index.tolist()

        if self.low_variance_features_:
            print(f"Dropping low-variance features (threshold={self.threshold}): {self.low_variance_features_}")
        else:
            print(f"No low variance features dropped (threshold={self.threshold})")

        self.feature_names_out_ = [col for col in X.columns if col not in self.low_variance_features_]
        return self

    def transform(self, X):
        X = X.copy()
        return X.drop(columns=self.low_variance_features_, errors='ignore')

    def get_feature_names_out(self, input_features=None):
        return self.feature_names_out_

class CorrelationReporter(BaseEstimator, TransformerMixin):
    def __init__(self, numeric_cols, threshold=0.9):
        self.numeric_cols = numeric_cols
        self.threshold = threshold
        self._transform_output = None

    def fit(self, X, y=None):
        X = X.copy()

        X_numeric = X[self.numeric_cols].select_dtypes(include='number')

        corr_matrix = X_numeric.corr(method='pearson').abs()

        upper = corr_matrix.where(
            np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
        )

        self.high_corr_pairs_ = [
            (col1, col2, upper.loc[col2, col1])
            for col1 in upper.columns
            for col2 in upper.index
            if not pd.isna(upper.loc[col2, col1]) and upper.loc[col2, col1] >= self.threshold
        ]

        if self.high_corr_pairs_:
            print(f"\n🔍 Highly correlated feature pairs (|r| ≥ {self.threshold}):")
            for a, b, r in sorted(self.high_corr_pairs_, key=lambda x: -x[2]):
                print(f"  {a} ↔ {b} | r = {r:.4f}")
        else:
            print(f"\n✅ No feature pairs with |r| ≥ {self.threshold}")

        return self

    def transform(self, X):
        return X

    def set_output(self, transform=None):
        self._transform_output = transform
        return self

numeric_pipeline = Pipeline([
    ('float', CoerceToFloat(columns=numeric_features)),
    ('imputer', NumericImputer(columns=numeric_features)),
    ('low_variance', LowVarianceDropper(threshold=0.001)),
    ('correlation_report', CorrelationReporter(numeric_cols=numeric_features, threshold=0.9)),
])

numeric_pipeline.set_output(transform="pandas")
_ = numeric_pipeline

#Pipeline
I ran Optuna with 30 trials using 5-fold cross-validation, optimizing specifically for recall, which was the primary objective. The best hyperparameters found from this study were hard-coded into the final model. Further optimization will be explored in future iterations to maximize performance further.

In [386]:
preprocessor = ColumnTransformer([
    ('bin', binary_pipeline, binary_cols),
    ('num', numeric_pipeline, numeric_features)
])

preprocessor.set_output(transform='pandas')

best_rf = RandomForestClassifier(
    n_estimators=238,
    max_depth=8,
    min_samples_split=6,
    min_samples_leaf=7,
    max_features='sqrt',
    class_weight='balanced',
    random_state=42
)
full_pipeline = Pipeline([
    ('feature_engineering', FeatureEngineer()),
    ('preprocessor', preprocessor),
    ('classifier', best_rf)
])

full_pipeline.set_output(transform="pandas")
_ = full_pipeline

#Metrics

In [387]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc'
}

scores = cross_validate(
    estimator=full_pipeline,
    X=X_train,
    y=y_train,
    cv=cv,
    scoring=scoring,
    return_train_score=False
)

cv_results = pd.DataFrame(scores)

print("Cross-Validation Performance:")
for metric in scoring:
    mean_score = cv_results[f'test_{metric}'].mean()
    std_score = cv_results[f'test_{metric}'].std()
    print(f"{metric:>10}: {mean_score:.4f} ± {std_score:.4f}")


No constant features dropped.
No low variance features dropped (threshold=0.001)

✅ No feature pairs with |r| ≥ 0.9
No constant features dropped.
No low variance features dropped (threshold=0.001)

✅ No feature pairs with |r| ≥ 0.9
No constant features dropped.
No low variance features dropped (threshold=0.001)

✅ No feature pairs with |r| ≥ 0.9
No constant features dropped.
No low variance features dropped (threshold=0.001)

✅ No feature pairs with |r| ≥ 0.9
No constant features dropped.
No low variance features dropped (threshold=0.001)

✅ No feature pairs with |r| ≥ 0.9
Cross-Validation Performance:
  accuracy: 0.8704 ± 0.0166
 precision: 0.7699 ± 0.0392
    recall: 0.8592 ± 0.0793
        f1: 0.8093 ± 0.0293
   roc_auc: 0.9324 ± 0.0471


# Summary after tuning and feature engineering
1. No constant features were found among the binary features.

2. No low-variance features were identified among the continuous numeric features.

3. No feature correlations exceeded the 0.9 threshold, indicating low multicollinearity.

After tuning and feature engineering the cross-validation performance improved significantly across all key metrics. Most notably, recall increased from 0.70 to 0.85, with a more consistent standard deviation — aligning well with the primary objective of improving sensitivity.